In [1]:
import numpy as np
import pandas as pd

df = pd.read_csv("../OpenAi/amlo_clasify_chatpgt3.csv")

In [2]:
import spacy 
from spacy import displacy 

nlp = spacy.load("es_core_news_lg")
import nltk
import re
from nltk import word_tokenize
from nltk.corpus import stopwords

stop_words_en= stopwords.words('spanish')
from nltk.stem.wordnet import WordNetLemmatizer

lemmatizer = WordNetLemmatizer()

In [3]:
def clean(texto):
    texto = texto.lower()
    texto = re.sub(r'([^0-9A-Za-z-À-ÿ \t])','', texto)
    texto = word_tokenize(texto)
    texto = [palabra for palabra in texto if palabra not in stop_words_en]
    texto = [lemmatizer.lemmatize(palabra) for palabra in texto]
    texto = ' '.join(texto)
    return texto

def vectorize_clean(texto):
    texto = texto.lower()
    texto = re.sub(r'([^0-9A-Za-z-À-ÿ \t])','', texto)
    texto = word_tokenize(texto)
    texto = [palabra for palabra in texto if palabra not in stop_words_en]
    texto = [lemmatizer.lemmatize(palabra) for palabra in texto]
    texto = ' '.join(texto)
    texto = nlp(texto).vector
    return texto

def vectorize(texto):
    texto = nlp(texto).vector
    return texto


    

In [4]:
type(df['vector'][0])

str

In [5]:
df['vector'] = df["vector"].apply(lambda x: 
                           np.fromstring(
                               x.replace('\n','')
                                .replace('[','')
                                .replace(']','')
                                .replace('  ',' '), sep=' '))


df['vector'] = df["clean"].apply(vectorize)
df


In [6]:
def getTopXDocs_large(frase,x,export=False):
    data = {
        'texto' :[],
        'sims': []
    }
    buscar = vectorize_clean(frase)
    for vector, frase in zip(df["vector"],df["Texto"]):
        A = buscar 
        B = vector
        resultado = np.dot(A,B) / (np.linalg.norm(A) * np.linalg.norm(B)) 
        
        data["texto"].append(frase)
        data["sims"].append(resultado)
    final = pd.DataFrame(data).sort_values(by = 'sims', ascending = False).head(x)
    return final

In [8]:
getTopXDocs_large("hubo muchas muertes", 25)

,texto,sims
15723,"Sí, sí. Y se les enfrenta y hay muertos, las ...",0.694906
3697,"Y hay muchas cosas, ¿no? Otras, de que una tra...",0.682967
4758,Eso es lo que se tiene que investigar para qu...,0.676752
11079,"Le traía pitayas, también les mandaron pitayas...",0.673640
1151,¿Por qué los mexicanos hemos enfrentado todo t...,0.661225
5375,"En ese entonces meten las armas, desde luego q...",0.659474
18269,"Sí, sí. Pero yo estoy seguro, pues, que se le...",0.656725
17745,"Sin embargo, no deja de haber un sector, que n...",0.651915
8605,"Entonces, la gente desconoce muchas cosas, aho...",0.648496
14253,Ahora lo mismo. Me siento muy orgulloso de que...,0.646816


In [153]:
df.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21726 entries, 0 to 21725
Data columns (total 6 columns):
 #   Column                  Non-Null Count  Dtype 
---  ------                  --------------  ----- 
 0   Texto                   21726 non-null  object
 1   Clasificacion           477 non-null    object
 2   new_classification      21726 non-null  object
 3   classification_spanish  21726 non-null  object
 4   clean                   21726 non-null  object
 5   vector                  21726 non-null  object
dtypes: object(6)
memory usage: 1018.5+ KB
